In [3]:
!pip install -q transformers==4.41.2 datasets==2.20.0 torch==2.3.1 accelerate==0.31.0
!pip install -U transformers==4.44.2 accelerate==0.34.2
# !pip install -U "numpy<2" datasets==2.20.0
!pip install -U "numpy<2" torch==2.3.1 transformers==4.44.2 datasets==2.20.0 accelerate==0.34.2


  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.31.0
    Uninstalling accelerate-0.31.0:
      Successfully uninstalled accelerate-0.31.0


In [5]:



from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# 1️⃣ Load tokenizer first
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

# 2️⃣ Load model
model = GPT2LMHeadModel.from_pretrained("distilgpt2")
model.resize_token_embeddings(len(tokenizer))

# 3️⃣ Tiny dataset for testing (replace with yours)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
tokenized_ds = dataset.map(tokenize_function, batched=True)
tokenized_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

# 4️⃣ Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 5️⃣ Training args
training_args = TrainingArguments(
    output_dir="./distilgpt2-finetuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=1,  # smaller batch
    num_train_epochs=1,
    save_strategy="no",
    logging_steps=10,
    report_to="none"
)

# 6️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator
)

# 7️⃣ Train
trainer.train()


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [5]:

# # 9️⃣ Evaluate
# print("\n📊 Evaluating...")
# eval_results = trainer.evaluate()
# perplexity = math.exp(eval_results["eval_loss"])
# print(f"Perplexity: {perplexity:.2f}")

# 🔟 Save locally
print("\n💾 Saving model...")
trainer.save_model("./distilgpt2-finetuned")
tokenizer.save_pretrained("./distilgpt2-finetuned")


💾 Saving model...


('./distilgpt2-finetuned/tokenizer_config.json',
 './distilgpt2-finetuned/special_tokens_map.json',
 './distilgpt2-finetuned/vocab.json',
 './distilgpt2-finetuned/merges.txt',
 './distilgpt2-finetuned/added_tokens.json')

In [7]:

# 11️⃣ Generate sample text
prompt = "In the near future, Egypt will eat"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=60,
    temperature=0.8,
    top_p=0.9,
    do_sample=True
)
print("\n📝 Generated text:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📝 Generated text:
 In the near future, Egypt will eat the land of the Pharaoh, a land that was previously ruled by the Egyptian Pharaohs, and the Sinai, as the latter was a land that was previously ruled by the Egyptian rulers. The land of the Pharaoh, Egypt, was originally a land of land of the Egyptian rulers, and was not a


In [ ]:

# 12️⃣ Optional: Upload to Hugging Face Hub
upload = input("\n🌐 Do you want to upload this model to Hugging Face? (y/n): ").strip().lower()
if upload == "y":
    login()  # paste your token
    trainer.push_to_hub("your-username/gpt2-finetuned-demo")
    tokenizer.push_to_hub("your-username/gpt2-finetuned-demo")
    print("✅ Uploaded to Hugging Face Hub!")
else:
    print("❎ Skipping upload. Model saved locally.")

In [ ]:
# test me